# Objectifs

* ~~capture multiples photos sur une durée précise~~ capture video ok -> envoi numpy array au cnn
* ~~se séparer de l'écriture (=enregistrement) en local de ces photos (pas de photo.jpeg en local)~~ os.remove fonctionnel
* ~~parvenir à écrire de multiples frames ou les regrouper dans un batch à passer au cnn (sans écrire le batch en local)~~ inclus en vidéo
* ~~avoir un simple script module_1 qui prenne en input la photo, et la restitue en output~~ ok dummycnn.py
* **ignorer deepface: fouiller les outils openCV pour détecter la présence d'un visage** -> nice-to-have
* ~~supprimer l'enregistrement vidéo et simplifier la cellule openCV~~ ok: déclenchement/interruption manuel (lien avec case app début/fin exercice)
* ~~collectiviser les réponses CNN, extraire la plus fréquente,~~ envoyer au LLM cette réponse AVEC une limite dans le temps de nb de requêtes

In [1]:
import numpy
import cv2
from PIL import Image
import time
from collections import deque, Counter

In [ ]:
import testcnn

In [8]:
def showfps(frame, prev_frame_time):
    #parameters
    font = cv2.FONT_HERSHEY_SIMPLEX
    #calculate fps
    new_frame_time = time.time()
    fps = 1/(new_frame_time - prev_frame_time)
    fps = str(round(fps))
    cv2.putText(frame, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
    return new_frame_time

In [7]:
def get_response_from_cnn(frame):
    pilimage = Image.fromarray(frame).convert("RGB")
    cnnresponse = testcnn.testrestitution(pilimage)
    return cnnresponse

In [16]:
def evaluate_response(history):
    return Counter(history).most_common(1)[0][0]

In [23]:
def perform_action(action):
    print(f"[LLM] Message envoyé pour l’émotion détectée : {action}")

    if action == "fatigue":
        print("LLM: Tu sembles fatigué(e), une courte pause peut t’aider .")
    elif action == "annoyance":
        print("LLM: Respire un bon coup, tu fais de ton mieux .")
    elif action == "disconnection":
        print("LLM: Hey ! Reviens avec nous, tu peux le faire .")
    elif action == "doubt":
        print("Je suis là pour t'aider, prenons les choses une étape à la fois.")
    elif action == "pain":
        print("Prends une pause pour te reposer, et si la douleur persiste, consulte un médecin, d'accord ?")
    elif action == "disquietment":
        print("Je suis là pour toi, prenons les choses une étape à la fois.")

In [ ]:
#def
#evaluate -> (Counter) détermine l'émotion la plus fréquemment détectée -> retourne variable action
#action -> variable tirée d'evaluate, pour déterminer le thème du msg à envoyer au LLM (ou ne rien envoyer si "others");
#  
#trouver une variable pour espacer l'envoi de msg (surtout empêcher l'envoi) sur une durée précise (ex 1min)

In [25]:
cap = cv2.VideoCapture(0)
prev_frame_time = 0
new_frame_time = 0
deq_len = 100
history = deque([], maxlen=deq_len)
last_action_time = time.time()

while( cap.isOpened() ):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.flip(frame,1)
        prev_frame_time = showfps(frame, prev_frame_time)

        cnnresponse = get_response_from_cnn(frame)
        print(cnnresponse)
        history.append(cnnresponse)

        if len(history) == deq_len:
            action = evaluate_response(history)

            if time.time() - last_action_time >= 5:
                last_action_time = time.time()
                perform_action(action)

        cv2.imshow('frame' , frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

disconnection
disquietment
disquietment
fatigue
doubt
disquietment
disconnection
doubt
pain
pain
annoyance
doubt
pain
fatigue
other
doubt
fatigue
other
disquietment
annoyance
pain
doubt
disquietment
disconnection
pain
fatigue
other
disquietment
doubt
disquietment
doubt
doubt
disconnection
fatigue
doubt
annoyance
fatigue
disquietment
disconnection
annoyance
disquietment
disquietment
annoyance
pain
annoyance
annoyance
other
fatigue
other
other
annoyance
annoyance
fatigue
disquietment
other
annoyance
annoyance
fatigue
other
annoyance
fatigue
disconnection
disquietment
annoyance
pain
disquietment
fatigue
annoyance
pain
doubt
disconnection
pain
disconnection
doubt
other
annoyance
pain
pain
other
annoyance
pain
pain
annoyance
disconnection
annoyance
pain
pain
fatigue
doubt
other
doubt
doubt
fatigue
pain
pain
pain
annoyance
annoyance
pain
doubt
fatigue
other
doubt
annoyance
disquietment
fatigue
disconnection
fatigue
pain
other
fatigue
disquietment
annoyance
disquietment
disconnection
pain
dou

In [24]:
# Initialize webcam (0 = default camera)
cam = cv2.VideoCapture(0)

# Capture one frame
ret, frame = cam.read()

if ret:
    cv2.imwrite("captured_image.jpeg", frame)  
else:
    print("Failed to capture image.")

cam.release()

In [ ]:
%load_ext autoreload
%autoreload 2

2